## General Processing

Workflow:
1. General filter
2. Outliers detection
3. Train and test set splitting
4. Missing values imputation
5. Add Score values
6. Output the train and test data




### Load master dataset

In [1]:
import pandas as pd
import os
from thesis_helper import *
from sklearn.impute import SimpleImputer
from dataset_path import output_path,mimic_iv_path
path = output_path
df_master = pd.read_csv(os.path.join(path, 'master_dataset.csv'))

In [2]:
percent_missing = df_master.isnull().sum() * 100 / len(df_master)
missing_value_df = pd.DataFrame({'column_name': df_master.columns,
                                 'percent_missing': percent_missing})
missing_value_df.to_csv('c.csv',index = False)

In [5]:
df_master['subject_id'].nunique()

205504

In [4]:
df_master

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,...,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon
0,0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,WHITE,AMBULANCE,ADMITTED,...,0,97.7,79.0,16.0,98.0,107.0,60.0,0.0,0.0,9.0
1,1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,WHITE,AMBULANCE,ADMITTED,...,1,97.9,86.0,17.0,93.0,96.0,57.0,5.0,3.0,12.0
2,3,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,WHITE,AMBULANCE,HOME,...,1,98.2,85.0,18.0,98.0,81.0,38.0,0.0,2.0,14.0
3,4,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,WHITE,AMBULANCE,ADMITTED,...,1,99.0,96.0,18.0,97.0,86.0,45.0,NaN,8.0,14.0
4,2,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,WHITE,AMBULANCE,ADMITTED,...,1,98.1,91.0,18.0,99.0,98.0,60.0,NaN,6.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425082,425081,19999784,25715748.0,34149746,2119-08-11 09:15:00,2119-08-11 13:40:00,M,BLACK/AFRICAN AMERICAN,WALK IN,ADMITTED,...,0,98.1,72.0,16.0,100.0,131.0,92.0,0.0,1.0,8.0
425083,425084,19999828,29734428.0,30712109,2147-07-17 17:18:00,2147-07-18 17:34:00,F,WHITE,WALK IN,ADMITTED,...,0,99.6,78.0,17.0,99.0,108.0,87.0,9.0,11.0,8.0
425084,425083,19999828,25744818.0,32917002,2149-01-08 09:11:00,2149-01-08 18:12:00,F,WHITE,AMBULANCE,ADMITTED,...,0,98.1,109.0,15.0,96.0,111.0,78.0,4.0,8.0,11.0
425085,425085,19999914,NaN,32002659,2158-12-24 11:41:00,2158-12-24 11:56:00,F,UNKNOWN,UNKNOWN,ELOPED,...,0,99.5,81.0,10.0,100.0,93.0,55.0,0.0,5.0,0.0


In [4]:
pd.set_option('display.max_columns', 200) 
df_master.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcomplaint,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon
0,0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,Other,2180,52,False,0 days 04:13:00,NaN,NaN,False,True,False,0,0,0,2180-06-26 15:54:00,50 days 16:24:00,False,0,0,0,0,0,0,4.22,NaN,50.68,98.4,70.0,16.0,97.0,106.0,63.0,0.0,3.0,"Abd pain, Abdominal distention",False,True,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,97.7,79.0,16.0,98.0,107.0,60.0,0.0,0.0,9.0
1,1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,Medicaid,2180,52,False,0 days 05:37:00,NaN,NaN,False,True,False,0,1,1,2180-07-22 16:24:00,25 days 18:53:00,False,0,1,1,0,0,0,5.62,NaN,25.79,98.9,88.0,18.0,97.0,116.0,88.0,10.0,3.0,Abdominal distention,False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,97.9,86.0,17.0,93.0,96.0,57.0,5.0,3.0,12.0
2,3,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,WHITE,AMBULANCE,HOME,52,2180,2180-09-09,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,Medicaid,2180,52,False,0 days 13:30:00,2180-07-23 14:00:00,0 days 08:06:00,True,True,True,1,2,2,2180-07-23 05:54:00,0 days 00:00:00,True,1,2,2,0,0,0,13.50,8.1,0.00,97.8,87.0,14.0,97.0,71.0,43.0,7.0,2.0,Hypotension,False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,98.2,85.0,18.0,98.0,81.0,38.0,0.0,2.0,14.0
3,4,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,Medicaid,2180,52,False,0 days 08:06:00,2180-07-23 14:00:00,0 days 00:00:00,True,True,True,2,3,3,2180-08-05 20:58:00,13 days 06:58:00,False,1,2,2,0,0,0,8.10,0.0,13.29,98.7,77.0,16.0,98.0,96.0,50.0,13.0,2.0,"Abdominal distention, Abd pain, LETHAGIC",False,True,False,False,False,False,False,False,False,False,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1

### 1. General filter - Age, triage_acuity

In [5]:
print('Before filtering for "age" >= 18 : master dataset size = ', len(df_master))
df_master = df_master[df_master['age'] >= 18]
print('After filtering for "age" >= 18 : master dataset size = ', len(df_master))

Before filtering for "age" >= 18 : master dataset size =  425087
After filtering for "age" >= 18 : master dataset size =  425087


In [6]:
# na_master = df_master[df_master['triage_acuity'].isnull()]
# len(na_master)

In [7]:
print('Before filtering for non-null "triage_acuity" >= 18 : master dataset size = ', len(df_master))
df_master = df_master[df_master['triage_acuity'].notnull()]
print('After filtering for non-null "triage_acuity" >= 18 : master dataset size = ', len(df_master))

Before filtering for non-null "triage_acuity" >= 18 : master dataset size =  425087
After filtering for non-null "triage_acuity" >= 18 : master dataset size =  418100


### 2. Outlier Detection 

In [6]:
# from mimic-extract
vitals_valid_range = {
    'temperature': {'outlier_low': 14.2, 'valid_low': 26, 'valid_high': 45, 'outlier_high':47},
    'heartrate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 350, 'outlier_high':390},
    'resprate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 300, 'outlier_high':330},
    'o2sat': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 100, 'outlier_high':150},
    'sbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'dbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'pain': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 10, 'outlier_high':10},
    'acuity': {'outlier_low': 1, 'valid_low': 1, 'valid_high': 5, 'outlier_high':5},
}

In [7]:
df_master = convert_temp_to_celcius(df_master)

In [8]:
display_outliers_count(df_master, vitals_valid_range)

C:\Users\DavidGu\Downloads\mimic4ed-benchmark-main\Benchmark_scripts\helpers.py:444: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  display_df = display_df.append({'variable': column,
C:\Users\DavidGu\Downloads\mimic4ed-benchmark-main\Benchmark_scripts\helpers.py:444: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  display_df = display_df.append({'variable': column,
C:\Users\DavidGu\Downloads\mimic4ed-benchmark-main\Benchmark_scripts\helpers.py:444: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  display_df = display_df.append({'variable': column,
C:\Users\DavidGu\Downloads\mimic4ed-benchmark-main\Benchmark_scripts\helpers.py:444: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

,variable,< outlier_low,"[outlier_low, valid_low)","[valid_low, valid_high]","(valid_high, outlier_high]",> outlier_high
0,triage_temperature,479,29,400985,0,9
1,triage_heartrate,0,0,407806,0,7
2,triage_resprate,0,0,404552,0,3
3,triage_o2sat,0,0,404261,6,41
4,triage_sbp,0,0,406601,0,16
5,triage_dbp,0,0,405422,0,395
6,triage_pain,0,0,386674,0,11193
7,triage_acuity,0,0,418100,0,0
8,ed_temperature_last,635,43,395351,1,47
9,ed_heartrate_last,0,0,403322,0,0


In [9]:

df_master = remove_outliers(df_master, vitals_valid_range)


### 3. Dataset Split (train:0.8, test: 0.2, use seed to fix)

In [10]:
df_train=df_master.sample(frac=0.8,random_state=10) #random state is a seed value
df_test=df_master.drop(df_train.index)

In [11]:
print('Training dataset size = ', len(df_train))
print('Testing dataset size = ', len(df_test))

Training dataset size =  334480
Testing dataset size =  83620


In [12]:
df_train.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcomplaint,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon
183244,183244,14328106,NaN,32336955,2159-11-08 15:13:00,2159-11-09 01:47:00,F,WHITE,WALK IN,HOME,75,2159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159,75,False,0 days 10:34:00,NaN,NaN,False,False,False,0,0,0,NaN,NaN,False,0,0,0,0,0,0,10.57,NaN,NaN,38.055556,65.0,18.0,100.0,135.0,59.0,0.0,2.0,"Dizziness, Unsteady gait",False,False,False,False,False,False,False,False,False,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.444444,53.0,16.0,98.0,130.0,68.0,0.0,5.0,9.0
376935,376935,18860711,NaN,35614382,2159-10-24 06:19:00,2159-10-24 14:31:00,F,WHITE,WALK IN,HOME,28,2159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159,28,False,0 days 08:12:00,NaN,NaN,False,False,False,1,1,1,NaN,NaN,False,0,0,0,0,0,0,8.20,NaN,NaN,36.500000,89.0,18.0,97.0,103.0,46.0,10.0,3.0,"Abd pain, n/v/d",False,True,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.722222,83.0,16.0,99.0,112.0,51.0,4.0,1.0,5.0
291574,291579,16860825,NaN,37485359,2203-01-07 17:45:00,2203-01-08 07:00:00,F,BLACK/AFRICAN AMERICAN,AMBULANCE,HOME,52,2194,NaN,NaN,NaN,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,NaN,2203,61,False,0 days 13:15:00,NaN,NaN,False,False,False,0,0,5,2203-01-12 22:26:00,4 days 15:26:00,False,0,0,2,0,0,0,13.25,NaN,4.64,36.888889,96.0,16.0,96.0,160.0,80.0,9.0,3.0,"Abd pain, Nausea, Diarrhea",False,True,False,False,False,False,True,False,False,False,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,36.722222,70.0,18.0,98.0,140.0,94.0,3.0,0.0,14.0
210522,210524,14971343,29116455.0,34783406,2207-07-11 18:24:00,2207-07-11 22:37:00,F,BLACK/AFRICAN AMERICAN,WALK IN,ADMITTED,74,2198,NaN,2207-07-11 20:24:00,2207-07-12 19:15:00,NaN,BLACK/AFRICAN AMERICAN,2207-07-11 18:24:00,2207-07-11 22:37:00,Medicare,2207,83,False,0 days 04:13:00,NaN,NaN,False,True,False,0,0,0,2208-08-28 08:10:00,413 days 09:33:00,False,0,0,0,0,0,0,4.22,NaN,413.40,36.944444,104.0,16.0,100.0,172.0,89.0,0.0,2.0,"Anxiety, Palpitations",False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,36.666667,83.0,14.0,99.0,136.0,77.0,0.0,3.0,5.0
69270,69316,11648387,NaN,39707189,2181-01-23 09:25:00,2181-01-23 13:27:00,M,WHITE,WALK IN,HOME,64,2175,2183-08-25,NaN,NaN,NaN,WHITE,NaN,NaN,NaN,2181,70,False,0 days 04:02:00,Na

### 4. Missing Value imputation 

In [13]:
df_missing_stats = df_train.isnull().sum().to_frame().T
df_missing_stats.loc[1] = df_missing_stats.loc[0] / len(df_train)
df_missing_stats.index = ['no. of missing values', 'percentage of missing values']
df_missing_stats

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcomplaint,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon
no. of missing values,0.0,0.0,176213.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,285030.000000,176213.000000,176213.000000,331376.00000,106901.000000,176213.000000,176213.000000,176213.000000,0.0,0.0,0.0,0.0,311505.000000,311505.000000,0.0,0.0,0.0,0.0,0.0,0.0,160584.0000,160584.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,311505.000000,160584.0000,13597.000000,8151.000000,10759.000000,11000.000000,9136.000000,10070.000000,25169.000000,0.0,14.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18132.00000,11802.000000,12189.000000,20325.000000,11981.00000,12175.0000,35845.000000,0.0,0.0
percentage of missing values,0.0,0.0,0.526827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.852159,0.526827,0.526827,0.99072,0.319604,0.526827,0.526827,0.526827,0.0,0.0,0.0,0.0,0.931311,0.931311,0.0,0.0,0.0,0.0,0.0,0.0,0.4801,0.4801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.931311,0.4801,0.040651,0.024369,0.032166,0.032887,0.027314,0.030106,0.075248,0.0,0.000042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05421,0.035285,0.036442,0.060766,0.03582,0.0364,0.107166,0.0,0.0


In [14]:
vitals_cols = [col for col in df_master.columns if len(col.split('_')) > 1 and 
                                                   col.split('_')[1] in vitals_valid_range and
                                                   col.split('_')[1] != 'acuity']
vitals_cols

['triage_temperature',
 'triage_heartrate',
 'triage_resprate',
 'triage_o2sat',
 'triage_sbp',
 'triage_dbp',
 'triage_pain',
 'ed_temperature_last',
 'ed_heartrate_last',
 'ed_resprate_last',
 'ed_o2sat_last',
 'ed_sbp_last',
 'ed_dbp_last',
 'ed_pain_last']

In [15]:
imputer = SimpleImputer(strategy='median')
df_train[vitals_cols] = imputer.fit_transform(df_train[vitals_cols])
df_test[vitals_cols] = imputer.transform(df_test[vitals_cols])

### 5. Add Score values

In [16]:
# test
add_triage_MAP(df_test) # add an extra variable MAP
add_score_CCI(df_test)
add_score_CART(df_test)
add_score_REMS(df_test)
add_score_NEWS(df_test)
add_score_NEWS2(df_test)
add_score_MEWS(df_test)

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


In [17]:
# train
add_triage_MAP(df_train) # add an extra variable MAP
add_score_CCI(df_train)
add_score_CART(df_train)
add_score_REMS(df_train)
add_score_NEWS(df_train)
add_score_NEWS2(df_train)
add_score_MEWS(df_train)

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


### 6. Output the train and test data

In [18]:
df_train.to_csv(os.path.join(path, 'train.csv'), index=False)
df_test.to_csv(os.path.join(path, 'test.csv'), index=False)

## Time series data processing

In [22]:
#mimic_iv_ed_path = 'your_path/mimic-iv-ed-1.0/ed'
df_vitalsign = read_vitalsign_table(os.path.join(mimic_iv_path, 'ed/vitalsign.csv.gz').replace("\\","/"))

In [23]:
df_vitalsign['charttime'] = pd.to_datetime(df_vitalsign['charttime'])
df_vitalsign.sort_values('charttime', inplace=True)
df_vitalsign.drop('ed_rhythm', axis=1, inplace=True)
df_vitalsign.head()

,subject_id,stay_id,charttime,ed_temperature,ed_heartrate,ed_resprate,ed_o2sat,ed_sbp,ed_dbp,ed_pain
503777,13238787,35341790,2110-01-11 01:49:00,98.4,77.0,16.0,100.0,149.0,104.0,8.0
836768,15350437,39042378,2110-01-11 03:45:00,97.1,71.0,16.0,100.0,117.0,79.0,0.0
503778,13238787,35341790,2110-01-11 04:02:00,98.0,78.0,18.0,99.0,138.0,92.0,0.0
836769,15350437,39042378,2110-01-11 05:42:00,98.4,57.0,16.0,100.0,132.0,90.0,0.0
836770,15350437,39042378,2110-01-11 08:10:00,99.1,83.0,18.0,100.0,152.0,97.0,0.0


In [24]:
grouped = df_vitalsign.groupby('stay_id')

In [25]:
resample_freq = '1H' # 1 hour
# resample_freq = '30T' # 30 minutes

df_list = []
counter = 0
N = len(grouped)
for stay_id, stay_df in grouped:
    counter += 1
    stay_df.set_index('charttime', inplace=True)
    stay_df = stay_df.resample('1T', origin='start').interpolate(method='linear')\
                     .resample(resample_freq, origin='start').asfreq().ffill().bfill()
    if counter % 10000 == 0 or counter == N:
        print('%d/%d' % (counter, N), end='\r')
    df_list.append(stay_df)

In [26]:
df_vitalsign_resampled = pd.concat(df_list)

In [27]:
df_vitalsign_resampled = convert_temp_to_celcius(df_vitalsign_resampled)

In [28]:
df_vitalsign_resampled = remove_outliers(df_vitalsign_resampled, vitals_valid_range)

In [29]:
vitals_cols = [col for col in df_vitalsign_resampled.columns if len(col.split('_')) > 1 and 
                                                                col.split('_')[1] in vitals_valid_range]
vitals_cols

['ed_temperature',
 'ed_heartrate',
 'ed_resprate',
 'ed_o2sat',
 'ed_sbp',
 'ed_dbp',
 'ed_pain']

In [30]:
imputer = SimpleImputer(strategy='median')
df_vitalsign_resampled[vitals_cols] = imputer.fit_transform(df_vitalsign_resampled[vitals_cols])

In [31]:
df_vitalsign_resampled.to_csv(os.path.join(output_path, 'ed_vitalsign_'+resample_freq+'_resampled.csv'))